In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt 
import sys
import os

from readCanLitefileUtils import *
from calibUtils import *

In [2]:
infile = '../TempCalibration/precalib_SN01_DACset000_100x.csv'
#f = open(infile)
#f.read()

#file_recordedValues = path+'boards_data - '+board+'_postcalib.csv'
#channel, DMM_000, DMM_3FF, DMM_CFF, DMM_FFF = np.loadtxt(infile, usecols=(0,1,2,3,4), delimiter=',', unpack='true')

#df_Tread = pd.read_csv(infile, usecols=["Length","Data (hex)"]) 
df_Tread = pd.read_csv(infile)
df_Tread.drop(['Bus','No', 'ID (hex)', 'Message', 'ASCII'], inplace=True, axis=1)


df_Tread       

tmp_filename = './Testfile.txt'
df_Tread.to_csv(tmp_filename, sep=',', index=False, header=False, quotechar = ' ')

#open file to read
infile = open(tmp_filename, 'r')

#call function which read the file and return the content ordered in a list of dictionaries
d_list = readfile(infile)



In [3]:
d_list

[{'timestamp': '338.104.652.0',
  'state': 'S',
  'mssg_length': '2',
  'mssg': '0105',
  'answer': ['01  E6  C4  E6  F2  E6  E7',
   '21  E6  D0  E6  CB  E6  E1',
   '41  E6  E0  E6  CD  E6  D7',
   '61  E6  DB  E6  D7  E6  D3']},
 {'timestamp': '338.304.090.0',
  'state': 'S',
  'mssg_length': '2',
  'mssg': '0100',
  'answer': ['01  FD  85  00  00  00  00', '21  93  BF  00  00  00  00']},
 {'timestamp': '338.504.319.0',
  'state': 'S',
  'mssg_length': '2',
  'mssg': '0104',
  'answer': ['01  00  00  00  00  00  00',
   '21  00  00  00  00  00  00',
   '61  00  00  00  00  00  00',
   '41  00  00  00  00  00  00']},
 {'timestamp': '339.104.350.0',
  'state': 'S',
  'mssg_length': '2',
  'mssg': '0105',
  'answer': ['01  E6  C5  E6  F1  E6  E9',
   '21  E6  D0  E6  CB  E6  E0',
   '41  E6  DE  E6  CF  E6  D8',
   '61  E6  DD  E6  D7  E6  D0']},
 {'timestamp': '339.304.431.0',
  'state': 'S',
  'mssg_length': '2',
  'mssg': '0100',
  'answer': ['01  FD  84  00  00  00  00', '21  93  C

In [4]:
def mygetHV(mylist):

    mylist = mylist.copy()

    list_HV = []

    for d in mylist:
        
        if (d.get('mssg')=='0100'):
            answ = d.get('answer')
            print(answ)

            if (len(answ) !=2): abort

            for answ_items in answ:
                answ_items = answ_items.split(' ')
                print("******", answ_items[0])
                if (answ_items[0] == '01'):
                    print("[2]", answ_items[2], "  [4]", answ_items[4])
                    list_HV.append(answ_items[2]+answ_items[4])


    #print(list_HV)
    list_HV_dec = map(todec, list_HV )
    HV_avg = statistics.mean(list_HV_dec)
    HV_hex = tohex(HV_avg)


    return  HV_hex








In [5]:
def mygetTemp(mylist):

    mylist = mylist.copy()

    list_NTC1 = []
    list_NTC2 = []
    list_TEMP = []

    for d in mylist:
        
        if (d.get('mssg')=='0100'):
            answ = d.get('answer')
            print(answ)

            if (len(answ) !=2): abort

            for answ_items in answ:
                answ_items = answ_items.split(' ')
                print("******", answ_items[0])
                if (answ_items[0] == '01'):
                    print("NTC1 [6]", answ_items[6], "  [8]", answ_items[8])
                    list_NTC1.append(answ_items[6]+answ_items[6])
                    print("NTC1 [6]", answ_items[10], "  [8]", answ_items[12])
                    list_NTC2.append(answ_items[10]+answ_items[12])
                elif (answ_items[0] == '21'):
                    print("TEMP [2]", answ_items[2], "  [4]", answ_items[4])
                    list_TEMP.append(answ_items[2]+answ_items[4])


    #print(list_HV)
    list_NTC1_dec = map(todec, list_NTC1 )
    list_NTC2_dec = map(todec, list_NTC2 )
    list_TEMP_dec = map(todec, list_TEMP )
    
    NTC1_avg = statistics.mean(list_NTC1_dec)
    NTC2_avg = statistics.mean(list_NTC2_dec)
    TEMP_avg = statistics.mean(list_TEMP_dec)
    
    print('NTC1 avg: ', NTC1_avg, 'NTC2 avg: ', NTC2_avg, 'TEMP avg: ', TEMP_avg )
    #TEMP_hex = tohex(HV_avg)


    return  TEMP_avg, NTC1_avg, NTC1_avg


In [6]:
mygetTemp(d_list) 

['01  FD  85  00  00  00  00', '21  93  BF  00  00  00  00']
****** 01
NTC1 [6] 00   [8] 00
NTC1 [6] 00   [8] 00
****** 21
TEMP [2] 93   [4] BF
['01  FD  84  00  00  00  00', '21  93  C2  00  00  00  00']
****** 01
NTC1 [6] 00   [8] 00
NTC1 [6] 00   [8] 00
****** 21
TEMP [2] 93   [4] C2
['01  FD  83  00  00  00  00', '21  93  C1  00  00  00  00']
****** 01
NTC1 [6] 00   [8] 00
NTC1 [6] 00   [8] 00
****** 21
TEMP [2] 93   [4] C1
['01  FD  83  00  00  00  00', '21  93  C5  00  00  00  00']
****** 01
NTC1 [6] 00   [8] 00
NTC1 [6] 00   [8] 00
****** 21
TEMP [2] 93   [4] C5
['01  FD  83  00  00  00  00', '21  93  CF  00  00  00  00']
****** 01
NTC1 [6] 00   [8] 00
NTC1 [6] 00   [8] 00
****** 21
TEMP [2] 93   [4] CF
['01  FD  83  00  00  00  00', '21  93  C6  00  00  00  00']
****** 01
NTC1 [6] 00   [8] 00
NTC1 [6] 00   [8] 00
****** 21
TEMP [2] 93   [4] C6
['01  FD  83  00  00  00  00', '21  93  D5  00  00  00  00']
****** 01
NTC1 [6] 00   [8] 00
NTC1 [6] 00   [8] 00
****** 21
TEMP [2] 93  

(37882.81, 0, 0)